<a href="https://colab.research.google.com/github/worldterminator/mess/blob/main/trump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
file_path = '/content/drive/My Drive/trump.csv'
trump = pd.read_csv(file_path)

print(trump.head())

Mounted at /content/drive
   Durationinseconds                                      describephoto  \
0                 30                                                NaN   
1                142                                            rioting   
2                243   Terrorist acts against a duly elected government   
3                162        People are trying to overturn our democracy   
4                282  White supremacists under the guise of Qanon ar...   

                                          whynervous feedback     IP_country  \
0                                                NaN      NaN  United States   
1         Because it shows disregard for rule of law      NaN  United States   
2  Our president is encouraging a coup and has un...      NaN  United States   
3  I think they're trying to disregard the basis ...      NaN  United States   
4  It is worrying that the president has encourag...      NaN  United States   

   ID  
0  66  
1  67  
2  68  
3  69  
4 

# flag by minimum characters

In [12]:
trump.dropna(subset=['whynervous'], inplace=True)  # dropping rows without a response
# Basic text cleaning (lowercasing, removing punctuation)
trump['whynervous_clean'] = trump['whynervous'].str.lower().str.replace('[^\w\s]', '', regex=True)
#just doing this for the whynervous variable here
min_words = 3
trump['flag_invalid'] = trump['whynervous_clean'].apply(lambda x: len(x.split()) < min_words)
print(trump)

      Durationinseconds                                      describephoto  \
1                   142                                            rioting   
2                   243   Terrorist acts against a duly elected government   
3                   162        People are trying to overturn our democracy   
4                   282  White supremacists under the guise of Qanon ar...   
5                   243  The people in the photo are attempting to brea...   
...                 ...                                                ...   
1266               3196  They are being sprayed by police while seeming...   
1267                577                            Protesting Trumps loss.   
1268                331  taking their country back and taking action to...   
1269               1366        Reacting negatively to the election outcome   
1277                323                                  an act of terror.   

                                             whynervous      fe

In [13]:
summary_stats = trump['flag_invalid'].describe()
invalid_valid_counts = trump['flag_invalid'].value_counts()
print(summary_stats)
print(invalid_valid_counts)

count      1180
unique        2
top       False
freq       1157
Name: flag_invalid, dtype: object
False    1157
True       23
Name: flag_invalid, dtype: int64


In [15]:
# to get the IDs, filter rows where 'flag_invalid' is True and print their corresponding IDs
invalid_responses = trump[trump['flag_invalid'] == True]
invalid_response_ids = invalid_responses['ID'].unique()

print(invalid_response_ids)


[ 104  171  189  381  401  476  489  582  719  720  751  779  790  862
  991 1060 1072 1073 1095 1148 1156 1160 1224]


# flag by minimum words

In [21]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# download necessary NLTK data (*)
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# tokenize and clean text (remove punctuation and stop words)
def clean_tokenize(text):
    no_punctuation = text.translate(str.maketrans('', '', string.punctuation))
    # tokenizing
    tokens = word_tokenize(no_punctuation)
    # removing stop words and lowercasing
    return [word.lower() for word in tokens if word.lower() not in stop_words]

# apply the functions
trump['whynervous_tokens_clean'] = trump['whynervous_clean'].apply(clean_tokenize)

# now count the tokens again
trump['token_count_clean'] = trump['whynervous_tokens_clean'].apply(len)

# Filtering rows based on the new token count
data_filtered = trump[trump['token_count_clean'] >= 3]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
data_filtered.shape

(1121, 12)